In [2]:
# eagerx imports
import eagerx
import eagerx_interbotix.demo as demo

In [3]:
# Set parameters
rate = 10

# Create graph
graph = eagerx.Graph.create()

In [4]:
# Create box
box = demo.Box.make(
    "box",
    rate=rate,
    sensors=["position", "yaw"],
)
graph.add(box)

graph.connect(source=box.sensors.position, observation="pos")
graph.connect(source=box.sensors.yaw, observation="yaw")

pybullet build time: May 20 2022 19:44:17


In [5]:
# Create goal
goal = demo.BoxGoal.make(
    "goal",
    rate=rate,
    sensors=["position", "yaw"],
)
graph.add(goal)

graph.connect(source=goal.sensors.position, observation="pos_desired")
graph.connect(source=goal.sensors.yaw, observation="yaw_desired")

In [6]:
# Create arm
arm = demo.PandaArm.make(
    name="panda",
    rate=rate,
    sensors=["position", "velocity", "force_torque", "ee_pos", "ee_orn"],
    actuators=["vel_control"],
)
graph.add(arm)

# Connecting observations
graph.connect(source=arm.sensors.position, observation="joints")
graph.connect(source=arm.sensors.velocity, observation="velocity")
graph.connect(source=arm.sensors.force_torque, observation="force_torque")
graph.connect(source=arm.sensors.ee_pos, observation="ee_position")

In [7]:
# Create IK node
ik = demo.PandaIK.make(
    name="inverse_kinematics",
    rate=rate,
)
graph.add(ik)

# Create safety node
safe = demo.VelocityControl.make(
    name="safety",
    rate=rate,
    joint_names=arm.config.joint_names,
    joint_upper=arm.config.joint_upper,
    joint_lower=arm.config.joint_lower,
    vel_limit=[0.2 * vl for vl in arm.config.vel_limit],
)
graph.add(safe)

# Connecting goal
graph.connect(source=ik.outputs.dtarget, target=safe.inputs.goal)

# Connecting safety filter to arm
graph.connect(source=arm.sensors.position, target=safe.inputs.position)
graph.connect(source=arm.sensors.velocity, target=safe.inputs.velocity)
graph.connect(source=safe.outputs.filtered, target=arm.actuators.vel_control)

# Connect IK
graph.connect(source=arm.sensors.position, target=ik.inputs.current)
graph.connect(source=arm.sensors.ee_pos, target=ik.inputs.xyz)
graph.connect(source=arm.sensors.ee_orn, target=ik.inputs.orn)

# Connecting actions
graph.connect(action="dxyz", target=ik.inputs.dxyz)
graph.connect(action="dyaw", target=ik.inputs.dyaw)

In [8]:
from eagerx.backends.single_process import SingleProcess
backend = SingleProcess.make()

from eagerx_pybullet.engine import PybulletEngine
engine = PybulletEngine.make(rate=rate)

env = demo.BoxPushEnv.make(
    name="PandaEnv",
    rate=rate,
    graph=graph,
    engine=engine,
    backend=backend,
)
obs = env.reset()

In [9]:
graph.gui()

In [10]:
for i in range(5):
    env.reset()
    rewards = 0
    done = False
    while not done:
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        rewards += reward
    print(f"Sum of rewards: {rewards}")

Sum of rewards: -226.7367985212725
Sum of rewards: -256.36138437981356
Sum of rewards: -157.82485195498964
Sum of rewards: -148.7184200441145
Sum of rewards: -236.96278857078923


In [11]:
env.shutdown()

In [12]:
from pathlib import Path
import eagerx_interbotix


# Load graph
graph = eagerx.Graph.load(f"{demo.LOG_DIR}/graph.yaml")

# Initialize env
env = demo.BoxPushEnv.make(
    name="ViperEnv",
    rate=rate,
    graph=graph,
    engine=engine,
    backend=backend,
)
obs = env.reset()

In [13]:
import stable_baselines3 as sb
import numpy as np


model = sb.SAC.load(f"{demo.LOG_DIR}/rl_model_25000_steps", env, verbose=0)
print("Evaluating policy for different numbers of steps.")
episodic_reward = []
for steps in [25_000, 100_000, 200_000, 500_000, 800_000, 1_100_000, 1_400_000, 1_600_000]:
    i += 1
    print(f"--------------\nSteps: {steps}")
    # Load model parameters
    model.set_parameters(f"{demo.LOG_DIR}/rl_model_{steps}_steps")

    # Evaluate
    obs, done = env.reset(), False
    rewards = 0
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        rewards += reward
    episodic_reward.append(rewards)
    print(f"Mean episodic reward: {np.mean(episodic_reward)}")

/home/jelle/.cache/pypoetry/virtualenvs/eagerx-interbotix-jybkouFr-py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Evaluating policy for different numbers of steps.
--------------
Steps: 25000
Mean episodic reward: -147.15401845014162
--------------
Steps: 100000
Mean episodic reward: -149.49296384234538
--------------
Steps: 200000
Mean episodic reward: -144.5104079043291
--------------
Steps: 500000
Mean episodic reward: -127.26668400525602
--------------
Steps: 800000
Mean episodic reward: -118.3885392434141
--------------
Steps: 1100000
Mean episodic reward: -110.89532068501065
--------------
Steps: 1400000
Mean episodic reward: -102.34126560962272
--------------
Steps: 1600000
Mean episodic reward: -96.9815334013345


In [14]:
env.shutdown()

<p align="center">
  <img src="gif/box_pushing_pybullet.gif" width="40%"  alt="box_sim"/>
  <img src="gif/box_pushing_real.gif" width="40%"  alt="box_real"/>
</p>

<p align="center">
  <img src="gif/pendulum_sim.gif" width="40%"  alt="pendulum_sim"/> 
  <img src="gif/pendulum_real.gif" width="40%"  alt="pendulum_real"/>
</p>
<img src="gif/all.gif" width="80%" />